In [1]:
from pyscf import __config__
# Default of 3.0 is too small for 6-31G*
# Note, this *must* be run before importing cubge
setattr(__config__, 'cubegen_box_margin', 4.0)

from pyscf import dft
from pyscf.geomopt import berny_solver
from pyscf.tools import cubegen

import nglview as nv
from utils import *

In [2]:
mol = SMILES_to_Mole("O")

In [3]:
# https://pyscf.org/user/gto.html
mol.basis="6-31G*"
mol.verbose = 0
mol.build()

In [4]:
# https://pyscf.org/user/dft.html
method = dft.RKS(mol)
method.xc = 'b3lypg'
method.scf()

-76.40702334862985

In [5]:
# https://pyscf.org/user/geomopt.html
opt_mol = berny_solver.optimize(method)

In [6]:
method = dft.RKS(opt_mol)
method.xc = 'b3lypg'
method.scf()

-76.40702397201403

In [7]:
_ = cubegen.density(opt_mol, 'water_den.cube', method.make_rdm1(), resolution=(1/6))
_ = cubegen.mep(opt_mol, 'water_pot.cube', method.make_rdm1(), resolution=(1/6))

In [8]:
view = nv.NGLWidget()
view.stage.set_parameters(**{
    "clipNear": 0, "clipFar": 100, "clipDist": 1,
})

c0 = view.add_component('water_den.cube')
c1 = view.add_component('water_pot.cube')

c0.add_surface(color_scheme='volume', 
       isolevelType="value",
       isolevel = 0.002,
       color_volume=c1,
       colorScale='rwb',
       colorDomain=[ -0.05, 0.05 ],      
       opacity=0.8
)

view

NGLWidget()